# Welcome to Dataskillet!

In [1]:
import warnings
# Filter Modin userwarnings about defaulting to pandas implementations
warnings.filterwarnings('ignore', category=UserWarning)

In [2]:
import os
import modin.pandas as pd
import pandas

In [3]:
from dataskillet import DataSource

In [4]:
%load_ext autoreload
%autoreload 2

# Loading data

In [5]:
os.makedirs('testdrive_csvs', exist_ok=True)

In [6]:
!curl https://raw.githubusercontent.com/jasonchang0/kaggle-google-apps/master/google-play-store-apps/googleplaystore.csv -o testdrive_csvs/googleplaystore.csv
!curl https://raw.githubusercontent.com/jasonchang0/kaggle-google-apps/master/google-play-store-apps/googleplaystore_user_reviews.csv -o testdrive_csvs/googleplaystore_user_reviews.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1328k  100 1328k    0     0  2938k      0 --:--:-- --:--:-- --:--:-- 2932k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7489k  100 7489k    0     0  10.1M      0 --:--:-- --:--:-- --:--:-- 10.1M


## The dataset contains google play apps and user reviews

In [7]:
googleplaystore = pd.read_csv('testdrive_csvs/googleplaystore.csv')
googleplaystore.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [8]:
googleplaystore_user_reviews = pd.read_csv('testdrive_csvs/googleplaystore_user_reviews.csv')
googleplaystore_user_reviews.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


# Quick interfaces

## sql_query

`sql_query` takes as input the sql string and a dictionary of Pandas or Modin dataframes.

In [9]:
from dataskillet import sql_query

In [10]:
result_df = sql_query("SELECT App, Category, Rating FROM table1 WHERE Price = '0' LIMIT 5", 
                      from_tables={"table1": googleplaystore})
result_df

,app,category,rating
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1
1,Coloring book moana,ART_AND_DESIGN,3.9
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3


Multiple tables can be passed to perform joins and complex queries.

In [17]:
sql = """
SELECT app, avg(sentiment_polarity) as avg_sentiment_polarity
FROM (
    SELECT app, sentiment, sentiment_polarity 
    FROM googleplaystore INNER JOIN googleplaystore_user_reviews 
    ON googleplaystore.app = googleplaystore_user_reviews.app 
) sub
GROUP BY app
HAVING CAST(avg_sentiment_polarity as float) > 0.4
LIMIT 10
"""
result_df = sql_query(sql, from_tables={"googleplaystore": googleplaystore, 
                                        "googleplaystore_user_reviews": googleplaystore_user_reviews})
result_df

,app,avg_sentiment_polarity
0,10 Best Foods for You,0.470733
4,2018Emoji Keyboard 😂 Emoticons Lite -sticker&gif,0.449566
8,2RedBeans,0.412199
11,365Scores - Live Scores,0.438312
14,3D Live Neon Weed Launcher,0.568182
19,7 Minute Workout,0.410278
22,850 Sports News Digest,0.542860
43,APE Weather ( Live Forecast),0.432323
44,"APUS Launcher - Theme, Wallpaper, Hide Apps",0.447473
52,ASUS Sound Recorder,0.516771


## Pandas/Modin extensions


`dataskillet.extensions` adds the `.sql` method to Pandas and Modin dataframes when imported. 

This method takes only an sql query. It's useful when only one table is used in the query.

In [12]:
# Make sure to import it **AFTER** Pandas or Modin
import dataskillet.extensions

### Example with a Modin DataFrame

In [13]:
print(type(googleplaystore))
googleplaystore.sql("SELECT App, Category") # Alternatively: "SELECT App, Category FROM temp"

<class 'modin.pandas.dataframe.DataFrame'>


,app,category
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN
1,Coloring book moana,ART_AND_DESIGN
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN
3,Sketch - Draw & Paint,ART_AND_DESIGN
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN
...,...,...
10836,Sya9a Maroc - FR,FAMILY
10837,Fr. Mike Schmitz Audio Teachings,FAMILY
10838,Parkinson Exercices FR,MEDICAL
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE


### Example with a Pandas DataFrame

The operations are executed in Modin

In [14]:
googleplaystore_pandas = pandas.read_csv('testdrive_csvs/googleplaystore.csv')

print(type(googleplaystore_pandas))
googleplaystore_pandas.sql("SELECT App, Category")

<class 'pandas.core.frame.DataFrame'>


,app,category
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN
1,Coloring book moana,ART_AND_DESIGN
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN
3,Sketch - Draw & Paint,ART_AND_DESIGN
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN
...,...,...
10836,Sya9a Maroc - FR,FAMILY
10837,Fr. Mike Schmitz Audio Teachings,FAMILY
10838,Parkinson Exercices FR,MEDICAL
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE


# In-depth usage

Under the hood `sql_query` creates a DataSource, adds tables to it, executes the query, and then tears everything down. This is not very efficient if many queries have to be executed against the same tables. In that case it's better to use DataSource directly.

DataSource is the analogus of database. It stores information about tables, loads tables from disk when they are needed, handles caching. 
It accepts queries like a database would.

## Creating the DataSource
On creation, we need to specify a directory to store table metadata for the DataSource.

In [15]:
metadata_dir = 'testdrive_metadata'
os.mkdir(metadata_dir)

FileExistsError: [Errno 17] File exists: 'testdrive_metadata'

In [ ]:
ds = DataSource(metadata_dir=metadata_dir)

The metadata dir now stores information about tables. However we currently have no tables. Lets create some

In [ ]:
ds.query('SHOW TABLES')

## Creating tables

A `Table` is an abstraction over a dataframe. It loads the dataframe on-demand. On creation it applies simple preprocessings. The preprocessings are created once and stored in metadata, so they are always performed the same way when a dataframe is loaded.

In [ ]:
ds.query(f'CREATE TABLE ("testdrive_csvs/googleplaystore.csv")')

In [ ]:
ds.query(f'CREATE TABLE ("testdrive_csvs/googleplaystore_user_reviews.csv")')

In [ ]:
ds.query('SHOW TABLES')

Now that we have some tables, the information about them is stored in metadata. 

If we recreate the datasource using the same `metadata_dir`, **we don't need to add the tables again**.

In [ ]:
ds = DataSource(metadata_dir=metadata_dir)

In [ ]:
ds.query('SHOW TABLES')

If we need it, we can clear the metadata and recreate the DataSource using `DataSource.create_new(metadata_dir)`

## Querying

In [ ]:
ds.query('SELECT * FROM googleplaystore LIMIT 5')

In [ ]:
ds.query("SELECT app FROM googleplaystore LIMIT 5")

In [ ]:
ds.query("SELECT CAST(price as int) as price_int FROM googleplaystore WHERE price = '0' LIMIT 5")

In [ ]:
ds.query("SELECT app, category FROM googleplaystore WHERE price = '0'")

In [ ]:
ds.query("SELECT app, category, reviews FROM googleplaystore WHERE category = 'FAMILY' AND price = '0' ORDER BY app ASC LIMIT 100")

In [ ]:
ds.query("SELECT app, sentiment, sentiment_polarity FROM googleplaystore INNER JOIN googleplaystore_user_reviews ON googleplaystore.app = googleplaystore_user_reviews.app LIMIT 10")

In [ ]:
sql = """
SELECT app, avg(sentiment_polarity) as avg_sentiment_polarity
FROM (
    SELECT app, sentiment, sentiment_polarity 
    FROM googleplaystore INNER JOIN googleplaystore_user_reviews ON googleplaystore.app = googleplaystore_user_reviews.app 
) sub
GROUP BY app
HAVING CAST(avg_sentiment_polarity as float) > 0.4
LIMIT 10
"""
ds.query(sql)

In [ ]:
sql = """
SELECT count(app) as count_apps
FROM (
    SELECT app
    FROM googleplaystore 
) sub
"""
ds.query(sql)

In [ ]:
sql = """
SELECT category, count(app) as count_apps
FROM (
    SELECT category, app FROM googleplaystore 
) sub
GROUP BY category
ORDER BY count_apps DESC
LIMIT 10
"""
ds.query(sql)

In [ ]:
sql = """
SELECT count(category) as uniq_categories
FROM (
    SELECT DISTINCT category FROM googleplaystore 
) sub
"""
ds.query(sql)

In [ ]:
tdf = ds.query("SELECT app || '_app_name'   FROM googleplaystore LIMIT 5")
tdf[0]

In [ ]:
ds.query("SELECT upper(app), category FROM googleplaystore WHERE app LIKE '.*best.*' LIMIT 5")

In [ ]:
ds.query("SELECT app, sentiment FROM googleplaystore_user_reviews WHERE translated_review IS NULL")

In [ ]:
ds.query("SELECT app, sentiment FROM googleplaystore_user_reviews WHERE sentiment IN ('Positive', 'Negative')")

## Dataframe caching

Dataskillet caches dataframes in memory to speed up queries against the same tables.

First let's clear the cache

In [ ]:
ds.cache.clear()

Querying with the cache clear

In [ ]:
%%time
_ = ds.query('SELECT * FROM googleplaystore LIMIT 5')

Check the cache

In [ ]:
ds.cache.get.cache_info()

Querying with the cache filled

In [ ]:
%%time
_ = ds.query('SELECT * FROM googleplaystore LIMIT 5')

Ensuring it was retrieved from cache:

In [ ]:
ds.cache.get.cache_info()

In [ ]:
%%timeit
_ = ds.query('SELECT * FROM googleplaystore LIMIT 5')